!pip install geopandas

### import required libraries

In [ ]:
# =========================
# 1. Import Libraries
# =========================
import requests
import json
import pandas as pd
import geopandas as gpd
from bs4 import BeautifulSoup
from shapely import wkt
from shapely.geometry import Point, Polygon
from sqlalchemy import create_engine


### Extract, Transform and Load - Household, Planning Area, And Supermarket data to cloud-based PostgreSQL  

In [ ]:
# ----------------------------
# Database Connection
# ----------------------------
engine = create_engine('postgresql://', echo=False)

# ----------------------------
# 1. Household Population ETL
# ----------------------------
def etl_population():
    dataset_id = "d_0ef47f84a0776b27010b242038ea2c6b"
    url = f"https://data.gov.sg/api/action/datastore_search?resource_id={dataset_id}"
    response = requests.get(url)
    records = response.json()['result']['records']

    # Transform
    df = pd.json_normalize(records)
    df.rename(columns={"Number": "area"}, inplace=True)
    df.drop(['_id'], axis=1, inplace=True)
    df.set_index('area', inplace=True)

    # Load
    df.to_sql('population', con=engine, if_exists='fail')
    print("Household Population ETL complete")
    return df

# ----------------------------
# 2.Planning Area ETL
# ----------------------------
def etl_planning_area():
    dataset_id = "d_4765db0e87b9c86336792efe8a1f7a66"
    url = f"https://api-open.data.gov.sg/v1/public/api/datasets/{dataset_id}/poll-download"
    response = requests.get(url)
    data_url = response.json()['data']['url']
    geojson_data = requests.get(data_url).json()

    # Transform
    gdf = gpd.GeoDataFrame.from_features(geojson_data['features'])
    
    # Extract area names from HTML Description
    for i in range(len(gdf)):
        soup = BeautifulSoup(gdf['Description'][i], 'html.parser')
        table = soup.find('table')
        first_row = table.find_all('tr')[1]
        area = first_row.find_all('td')[0].text.strip()
        gdf.loc[i, 'Description'] = area

    gdf = gdf.set_axis(['geometry','name','area'], axis=1)
    gdf['area'] = gdf['area'].str.title()
    gdf.set_index('area', inplace=True)
    gdf.drop(['name'], axis=1, inplace=True)

    # Load
    gdf.drop(columns=['geometry'], inplace=True)
    gdf.to_sql('planning_area', con=engine, if_exists='fail')
    print("Planning Area ETL complete")
    return gdf

# ----------------------------
# 3. Supermarkets ETL
# ----------------------------
def etl_supermarkets(planning_area_path='planning_area.csv'):
    dataset_id = "d_cac2c32f01960a3ad7202a99c27268a0"
    url = f"https://api-open.data.gov.sg/v1/public/api/datasets/{dataset_id}/poll-download"
    response = requests.get(url)
    data_url = response.json()['data']['url']
    geojson_data = requests.get(data_url).json()

    # Transform
    gdf = gpd.GeoDataFrame.from_features(geojson_data['features'])

    # Extract key values from Description table
    for i in range(len(gdf)):
        soup = BeautifulSoup(gdf['Description'][i], 'html.parser')
        table = soup.find('table')
        rows = table.find_all('tr')
        for row in rows[1:]:
            cols = row.find_all(['th','td'])
            if len(cols)==2:
                key = cols[0].text.strip()
                value = cols[1].text.strip()
                gdf.loc[i,key] = value

    gdf.drop(['Name','Description','LIC_NO','INC_CRC','FMEL_UPD_D'], axis=1, inplace=True)
    gdf.columns = gdf.columns.str.lower()

    # Spatial join: map each supermarket to planning area
    gdf['geometry'] = gdf['geometry'].apply(wkt.loads)
    planning_area = gpd.read_file(planning_area_path)
    planning_area['geometry'] = planning_area['geometry'].apply(wkt.loads)
    planning_area.set_index('area', inplace=True)

    for idx, row in gdf.iterrows():
        coord = row['geometry']
        for area_idx, area_row in planning_area.iterrows():
            if area_row['geometry'].contains(coord):
                gdf.loc[idx,'planning_area'] = area_idx
                break

    gdf.drop('geometry', axis=1, inplace=True)

    # Load
    gdf.to_sql('supermarkets', con=engine, if_exists='fail')
    print("Supermarkets ETL complete")
    return gdf

# ----------------------------
# Run all ETLs
# ----------------------------
if __name__ == "__main__":
    df_population = etl_population()
    df_planning = etl_planning_area()
    df_supermarkets = etl_supermarkets()